<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://supaerodatascience.github.io/machine-learning/">https://supaerodatascience.github.io/machine-learning/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Bayesian models for Machine Learning<br>Naive Bayes Classification</div>

One very common application of naive Bayes classifiers is document classification (e-mail spam filtering, sentiment analysis on social networks, technical documentation classification, customer appreciations, etc.). 

Naive Bayes classifiers for documents estimate the probability of a given document belonging to a certain class Y of documents, based on the document's contents Xi.


Suppose we want to predict the probability that sample $x$ has label $y$. This is a probability estimation problem that can be written:
$$\mathbb{P}(Y=y|X=x)$$

According to Bayes' theorem, we have:
$$\mathbb{P}(Y=y|X=x) =\frac{\mathbb{P}(X=x|Y=y)\cdot\mathbb{P}(Y=y)}{\mathbb{P}(X=x)}$$
$$\textrm{posterior} = \frac{\textrm{likelihood}\cdot\textrm{prior}}{\textrm{evidence}}$$

<div class="alert alert-success">
    
**Bayesian inference** is the problem of estimating this **posterior distribution**.<br>
In plain words, it consists in estimating the probability of label $y$, given an input $x$, using previously seen data to estimate the **likelihood** of an $x$ input associated to label $y$ and the general **prior** probability of observing label $y$.
</div>

Note that Bayesian inference applies both to classification and regression.

The goal of Bayesian inference is to estimate the label distribution for a given $x$ and use them to predict the correct label, so it is a *probabilistic approach to Machine Learning*.

The Bayesian predictor (classifier or regressor) returns the label that maximizes the posterior probability distribution.

In this (first) notebook on Bayesian modeling in ML, we will explore the method of Naive Bayes Classification.

1. [The naive Bayes assumption](#sec1)
2. [Naive Bayes classifiers in scikit-learn](#sec2)
3. [Examples](#sec3)
    1. [The "spam or ham?" example](#sec3-1)
    2. [The NIST example](#sec3-2)

# 1. <a id="sec1"></a>The naive Bayes assumption

Let's start with some illustrative data. We consider an artificial data set of 9 individuals. The first column in our data set is the sex ($S=0$ for male, 1 for female), the second is the height $H$ (in meters), the third is the weight $W$ (in kilos) and the last is the foot size $F$ (in centimeters).

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
fig_size=(10, 10)

data = np.loadtxt("sex_classif.csv")
data

array([[ 0.  ,  1.82, 82.  , 30.  ],
       [ 0.  ,  1.8 , 86.  , 28.  ],
       [ 0.  ,  1.7 , 77.  , 30.  ],
       [ 0.  ,  1.8 , 75.  , 25.  ],
       [ 1.  ,  1.52, 45.  , 15.  ],
       [ 1.  ,  1.65, 68.  , 20.  ],
       [ 1.  ,  1.68, 59.  , 18.  ],
       [ 1.  ,  1.75, 68.  , 23.  ],
       [ 1.  ,  1.58, 49.  , 19.  ]])

**Questions** :
- Using matplotlib, bokeh, seaborn or plotly, plot one relevant figure on the data

In [5]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
import numpy as np

# Exemple : Y = sexe (0=male,1=female)
Y = data[:, 0].astype(int)
X = data[:, 1:4]

# Pour affichage dans Jupyter
output_notebook()

# Préparer les données dans une ColumnDataSource
sex_labels = ["Homme", "Femme"]
colors = ["#1f77b4", "#ff7f0e"]
sex = np.array([sex_labels[int(i)] for i in Y])

source = ColumnDataSource(data=dict(
    H=X[:, 1],
    W=X[:, 2],
    sex=sex,
    color=[colors[i] for i in Y]
))

# Créer la figure
p = figure(
    title="Scatter plot H vs W selon le sexe",
    x_axis_label="H",
    y_axis_label="W",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    width=700,
    height=500
)

# Utiliser scatter() au lieu de circle()
p.scatter(
    x="H",
    y="W",
    size=8,
    color="color",
    alpha=0.7,
    legend_field="sex",
    source=source
)

# Personnaliser la légende
p.legend.title = "Sexe"
p.legend.location = "top_left"
p.legend.background_fill_alpha = 0.3
p.legend.border_line_alpha = 0.3

# Afficher le graphique
show(p)


Loading BokehJS ...

We would like to answer the question: is $(H=1.81, W=59, F=21)$ male or female?

Let's try to estimate $\mathbb{P}(S=0|H=1.81, W=59, F=21)$.

According to Bayes' theorem, the probability that a person that measures 1.81m, weights 59kgs and has a foot size of 21cm is male, is actually the likelihood of observing a person with such features among males, multiplied by the probability of observing males in the population, divided by the probability of observing an individual with these features.

That's a long sentence. Let's write that mathematically:
$$\mathbb{P}(S=0|H=1.81, W=59, F=21) = \frac{\mathbb{P}(H=1.81, W=59, F=21 | S=0)\cdot \mathbb{P}(S=0)}{\mathbb{P}(H=1.81, W=59, F=21)}$$

Let's make that more readable and more general:
$$\mathbb{P}(S|H, W, F) = \frac{\mathbb{P}(H,W,F | S)\cdot \mathbb{P}(S)}{\mathbb{P}(H,W,F)}$$

Interestingly, since our goal is only to compare the probabilities for $S=0$ and $S=1$, the denominator in the last equation won't be relevant. So we are left with two terms to estimate, given the available data:
- $\mathbb{P}(S=0)$: the prior - the probability that any individual is $S=0$, regardless of his/her physical attributes;
- $\mathbb{P}(H=1.81, W=59, F=21 | S=0)$: the likelihood of meeting somebody with the specified features, given that his/her sex is $S=0$.

The prior, in this case, is easy to estimate by comparing the frequencies of male and female individuals in the population.
\begin{gather*}
\mathbb{P}(S=0) = \frac{4}{9}\\
\mathbb{P}(S=1) = \frac{5}{9}
\end{gather*}
Technically, the estimate above is obtained by [*maximum likelihood estimation*](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation).

The likelihood, however, is a bit trickier. Can we directly estimate the **joint probability** of the 3 variables $(H,W,F)$?

Theoretically, we can. We can assume that among male individuals, $(H,W,F)$ are distributed according to a multivariate Normal distribution, with mean $\mu=(\mu_H, \mu_W, \mu_F)$ and covariance matrix $\Sigma$. The trick is then to estimate $\mu$ and $\Sigma$.

As a matter of fact, estimating $\mu$ and $\Sigma$ without further hypothesis would require quite a lot of data, especially because $\Sigma$ captures the **correlation** between $H$, $W$ and $F$.

$\Sigma$ is a $3\times 3$ matrix, so it involves 9 parameters to estimate, and we unfortunately only have 9 data points.

Let's rephrase this from another perspective. With some basic probabilities, we have:
\begin{align*}
\mathbb{P}(H,W,F | S) = &\mathbb{P}(H | S)\\
& \cdot \mathbb{P}(W | S, H) \\
& \cdot \mathbb{P}(F | S, H, W)
\end{align*}

Those three probabilities are univariate probabilities, much easier to estimate. However, the first one is a function of $S$ only, the second one depends on $S$ and $H$ and the third one depends on $S$, $H$ and $W$. To get an accurate estimate of the third one, we would need samples of the distribution of $F$ in enough points in the space of $(S,H,W)$ to cover it reasonably. This would require a number of data points that is exponential in the number of variables. That's what is called the **curse of dimensionality**, which makes this estimation problem difficult.

Let's make this concrete. Suppose we discretize $H$, $W$ and $F$ in 10 bins each and suppose we require 100 samples to get a correct estimate of $\mathbb{P}(F | S, H, W)$ for any given value of $(F, S, H, W)$. Then we need $100\cdot 10^3\cdot 2$ samples to correctly estimate this probability for all possible values of $(F, S, H, W)$. More generally, if we had $n$ continuous features rather than just three, we would require a number of data points that is exponential in $n$.

To circumvent this problem, we are going to make a very **naive** assumption (hence the name of the method). We are going to assume that the weight, the height and the foot size are totally independent variables, that is the probability that a person be 1.85m is the same whatever his/her weight and foot size.

Obviously, this hypothesis is very strong and clearly does not hold is most real-world cases. But we will assume it nonetheless. In this case, the likelihood estimation becomes:
\begin{align*}
\mathbb{P}(H,W,F | S) = &\mathbb{P}(H | S)\\
& \cdot \mathbb{P}(W | S) \\
& \cdot \mathbb{P}(F | S)
\end{align*}

Each of these probabilities now only depends on the label $S$ and is much easier to estimate from the data. This **conditional independence** assumption is called the **naive Bayes hypothesis**. It allow us to give a (very bad) estimate of $\mathbb{P}(X | Y)$ and hence of $\mathbb{P}(Y|X)$.

$$\mathbb{P}(S|H, W, F) = \frac{\mathbb{P}(H | S)\cdot \mathbb{P}(W | S) \cdot \mathbb{P}(F | S)\cdot \mathbb{P}(S)}{\mathbb{P}(H, W, F)}$$

Or, in our case:

$$\mathbb{P}(S=0|H=1.81, W=59, F=21) = \frac{\mathbb{P}(H=1.81 | S=0)\cdot \mathbb{P}(W=59 | S=0) \cdot \mathbb{P}(F=21 | S=0)\cdot \mathbb{P}(S=0)}{\mathbb{P}(H=1.81, W=59, F=21)}$$

The **naive Bayes classifier** is then the classifier that estimates all class probabilities and returns the one with maximum probability.

$$f(H, W, F) = \arg\max_{s} \mathbb{P}(S=s|H,W,F) = \arg\max_{s} \mathbb{P}(H|S=s)\cdot \mathbb{P}(W|S=s) \cdot \mathbb{P}(F|S=s)\cdot \mathbb{P}(S=s)$$

<div class="alert alert-warning">
    
**Exercice:**<br>
Let's implement a naive Bayes classifier on the data above, just to practice. We will assume that the $\mathbb{P}(X | S)$ distributions are Gaussians (for $X = H,W,$ or $F$). Compute the scores and probabilities for each sex, for $(H=1.81, W=59, F=21)$.<br>
Hint: use the `np.mean` and `np.std` functions to estimate distribution parameters. Use `scipy.stats.norm.pdf` to compute the Gaussian probability density function in a given input.
</div>

In [11]:
# %load solutions/code1.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

# Estimate distribution parameters for males
dataM = data[data[:,0]==0]
mu_HS0 = np.mean(dataM[:,1])
std_HS0 = np.std(dataM[:,1])
mu_WS0 = np.mean(dataM[:,2])
std_WS0 = np.std(dataM[:,2])
mu_FS0 = np.mean(dataM[:,3])
std_FS0 = np.std(dataM[:,3])
pS0 = dataM.shape[0]/data.shape[0]

# Estimate distribution parameters for females
dataF = data[data[:,0]==1]
mu_HS1 = np.mean(dataF[:,1])
std_HS1 = np.std(dataF[:,1])
mu_WS1 = np.mean(dataF[:,2])
std_WS1 = np.std(dataF[:,2])
mu_FS1 = np.mean(dataF[:,3])
std_FS1 = np.std(dataF[:,3])
pS1 = dataF.shape[0]/data.shape[0]

# score that (H=1.81,W=59,F=21) is male/female
H=1.81
W=59
F=21
from scipy.stats import norm
score_M = pS0 * norm.pdf(H,mu_HS0,std_HS0) * norm.pdf(W,mu_WS0,std_WS0) * norm.pdf(F,mu_FS0,std_FS0)
score_F = pS1 * norm.pdf(H,mu_HS1,std_HS1) * norm.pdf(W,mu_WS1,std_WS1) * norm.pdf(F,mu_FS1,std_FS1)
print("score male    :", score_M)
print("score female  :", score_F)
print("proba male    :", score_M/(score_M+score_F))
print("proba female  :", score_F/(score_M+score_F))


score male    : 6.981284895980458e-10
score female  : 0.0012161942837264688
proba male    : 5.740267802562792e-07
proba female  : 0.9999994259732197


It appears we will always multiply together values that are smaller than one. The result will quickly become very small. It is a good habit to move to log-scale.

<div class="alert alert-warning">

**Exercice:**<br>
Reuse your code above to compute log scores instead of scores.
</div>

In [14]:
# %load solutions/code2.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

log_score_M = np.log(pS0) + norm.logpdf(H,mu_HS0,std_HS0) + norm.logpdf(W,mu_WS0,std_WS0) + norm.logpdf(F,mu_FS0,std_FS0)
log_score_F = np.log(pS1) + norm.logpdf(H,mu_HS1,std_HS1) + norm.logpdf(W,mu_WS1,std_WS1) + norm.logpdf(F,mu_FS1,std_FS1)
print("log score male:    ", log_score_M)
print("log score female:  ", log_score_F)


log score male:     -21.08261794758859
log score female:   -6.712028735399105


Conclusion: $(H=1.81,W=59,F=21)$ is most probably female.

Let's generalize.

<div class="alert alert-success">
    
    
Given $n$ features $X_i$ and classes $Y$, **naive Bayes classifiers** estimate (from data) the distributions $\mathbb{P}(Y)$ and $\mathbb{P}(X_i|Y)$. Then, using Bayes rule and the naive Bayes assumption, they predict the most probable estimated class:
\begin{align*}
\arg\max_{y} \mathbb{P}(Y=y|X=x) & = \arg\max_{y} \frac{\prod\limits_{i=1}^n \mathbb{P}(X_i=x_i|Y=y) \mathbb{P}(Y=y)}{\mathbb{P}(X=x)}\\
& = \arg\max_{y} \prod\limits_{i=1}^n \mathbb{P}(X_i=x_i|Y=y) \mathbb{P}(Y=y)\\
& = \arg\max_{y} \sum\limits_{i=1}^n \log\left(\mathbb{P}(X_i=x_i|Y=y)\right) + \log\left(\mathbb{P}(Y=y)\right)
\end{align*}
</div>

Note that although it is not compulsory to compute the denominator, it is quite straightforward since:
\begin{align*}
\mathbb{P}(X=x) &= \sum\limits_y \mathbb{P}(X=x|Y=y)\mathbb{P}(Y=y)\\
&= \sum\limits_y \prod\limits_{i=1}^n \mathbb{P}(X_i=x_i|Y=y) \mathbb{P}(Y=y) 
\end{align*}
So it's the sum of the numerator's values for all $y$, so it's just a matter of normalizing the scores obtained.

A really nice thing about naive Bayes classifiers is that it is an **online method**, since most univariate probability distributions can be updated incrementally, 

# <a id="sec2"></a> 2. Naive Bayes classifiers in scikit-learn

Once again, scikit-learn has a [naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html) implementation. It allows three kind of distributions for the $X_i|Y$ variables: Normal (continuous), Bernouilli or Multinomial (discrete).
Let's directly use it on our toy dataset.

In [15]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
X = data[:,1:]
y = data[:,0]
gnb.fit(X,y)
xtest = np.array([[1.81,59,21]])
print("Prediction: ", gnb.predict(xtest))
print("Probas:     ", gnb.predict_proba(xtest))
print("Log probas: ",gnb.predict_log_proba(xtest))

Prediction:  [1.]
Probas:      [[5.73982396e-07 9.99999426e-01]]
Log probas:  [[-1.43706671e+01 -5.73982561e-07]]


# <a id="sec3"></a> 3. Examples

## <a id="sec3-1"></a> 3.1 The "spam or ham?" example

Let's scale up and apply naive Bayes classification on the ling-spam data. We will assume a multinomial distribution of word $i$ appearing in and email of class $y$.

In [16]:
from sys import path
path.append('../2 - Text data preprocessing')
import load_spam
spam_data = load_spam.spam_data_loader()
spam_data.load_data()
print("data loaded")

Xtrain, ytrain, Xtest, ytest = spam_data.split(2000)

/home/mohamed/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


data loaded


<div class="alert alert-warning">
    
**Exercice:**
Use scikit-learn to build a [multinomial naive Bayes classifier](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) on the data above. Estimate its generalization error.
</div>

In [27]:
# %load solutions/code3.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

from sklearn.naive_bayes import MultinomialNB
spam_nbc = MultinomialNB()
spam_nbc.fit(Xtrain,ytrain)
print("Score:", spam_nbc.score(Xtest,ytest))

# Compute cross-validation score
nb_trials = 20
score = []
for i in range(nb_trials):
    Xtrain, ytrain, Xtest, ytest = spam_data.shuffle_and_split(2000)
    spam_nbc = MultinomialNB()
    spam_nbc.fit(Xtrain,ytrain);
    score += [spam_nbc.score(Xtest,ytest)]
    print('*', end='')
print(" done!")
print("Average generalization score:", np.mean(score))
print("Standard deviation:", np.std(score))


Score: 0.9473684210526315
******************** done!
Average generalization score: 0.9232362821948487
Standard deviation: 0.008626800516698833


We've trained our model in the Tf-Idf data. Let's see how the model behaves on raw word counts.

<div class="alert alert-warning">

**Exercice:**
Use scikit-learn to build a [multinomial naive Bayes classifier](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) on the raw word counts data below. Estimate its generalization error.
</div>

In [19]:
Xtrain, ytrain, Xtest, ytest = spam_data.split(2000, feat='wordcount')

In [28]:
# %load solutions/code4.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).
from sklearn.naive_bayes import MultinomialNB
spam_nbc = MultinomialNB()
spam_nbc.fit(Xtrain,ytrain)
print("Score:", spam_nbc.score(Xtest,ytest))

# Compute cross-validation score
nb_trials = 20
score = []
for i in range(nb_trials):
    Xtrain, ytrain, Xtest, ytest = spam_data.shuffle_and_split(2000,feat='wordcount')
    spam_nbc = MultinomialNB()
    spam_nbc.fit(Xtrain,ytrain);
    score += [spam_nbc.score(Xtest,ytest)]
    print('*', end='')
print(" done!")
print("Average generalization score:", np.mean(score))
print("Standard deviation:", np.std(score))

Score: 0.9339305711086227
******************** done!
Average generalization score: 0.9882418812989922
Standard deviation: 0.0030972713742305146


Let's identify which are the misclassified emails (and find the confusion matrix by the way).

In [21]:
# Retrain
Xtrain, ytrain, Xtest, ytest = spam_data.split(2000, feat='wordcount')
spam_nbc = MultinomialNB()
spam_nbc.fit(Xtrain,ytrain);

In [22]:
# Find misclassified examples
ypredict = spam_nbc.predict(Xtest)
misclass = np.not_equal(ypredict, ytest)
Xmisclass = Xtest[misclass,:]
ymisclass = ytest[misclass]
misclass_indices = [i for i, j in enumerate(misclass) if j == True]
print("Misclassified messages indices:", misclass_indices)

Misclassified messages indices: [8, 100, 294, 383, 384, 450, 566, 598, 611, 628, 691, 725, 736, 737, 784, 807, 828]


In [23]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:")
print(confusion_matrix(ytest, ypredict))

Confusion matrix:
[[733  13]
 [  4 143]]


In [24]:
# Check some misclassified mails
index = misclass_indices[1]+2000
print("Prediction:", spam_nbc.predict(spam_data.word_count[index,:]))
spam_data.print_email(index)
spam_nbc.predict_proba(spam_data.tfidf[index,:])

Prediction: [False]
email file: ../data/lingspam_public/bare/part5/spmsgb164.txt
email is a spam: True
Subject: university degree programs

university degree programs increase your personal prestige and money earning power through an advanced university degree . eminent , non-accredited universities will award you a degree for only $ 200 . degree granted based on your present knowledge and experience . no further effort necessary on your part . just a short phone call is all that is required for a ba , ma , mba , or phd diploma in the field of your choice . for details , call 770-492 - 2925

Bag of words representation (28 words in dictionary):
{'subject': np.int64(1), 'university': np.int64(3), 'call': np.int64(2), 'part': np.int64(1), 'based': np.int64(1), 'phone': np.int64(1), 'money': np.int64(1), 'non': np.int64(1), 'power': np.int64(1), 'field': np.int64(1), 'experience': np.int64(1), 'necessary': np.int64(1), 'knowledge': np.int64(1), 'present': np.int64(1), 'short': np.int64(1)

array([[0.39310455, 0.60689545]])

**Questions** :
- What are the next steps to improve the results ? To create the moderation system ?
- What questions should you ask to the tech company, before working on their data ?

## <a id="sec3-2"></a> 3.2 The NIST example

We will assume Gaussian distributions for the NIST example.

In [29]:
from sklearn import datasets
%matplotlib inline
import matplotlib.pyplot as plt

digits = datasets.load_digits()
print(digits.data.shape)
print(digits.images.shape)
print(digits.target.shape)
print(digits.target_names)

X = digits.data
y = digits.target
Xtrain,Xtest = np.split(X,[1000])
ytrain,ytest = np.split(y,[1000])
#Xtrain = X[:1000,:]
#ytrain = y[:1000]
#Xtest = X[1000:,:]
#ytest = y[1000:]

#print(digits.DESCR)

#plt.gray();
#plt.matshow(digits.images[0]);
#plt.show();
#plt.matshow(digits.images[15]);
#plt.show();
#plt.matshow(digits.images[42]);
#plt.show();

from sklearn.utils import shuffle

def shuffle_and_split(X,y,n):
    X0,y0 = shuffle(X,y)
    Xtrain,Xtest = np.split(X0,[n])
    ytrain,ytest = np.split(y0,[n])
    return Xtrain, ytrain, Xtest, ytest

(1797, 64)
(1797, 8, 8)
(1797,)
[0 1 2 3 4 5 6 7 8 9]


In [30]:
Xtrain, ytrain, Xtest, ytest  = shuffle_and_split(X,y,1000)

print(Xtrain.shape)
print(ytrain.shape)
digits_nbc = GaussianNB()
digits_nbc.fit(Xtrain,ytrain)
prediction = digits_nbc.predict(Xtest)
#print("Training error:", np.sum(np.not_equal(prediction,ytest))/len(ytest))
print("Generalization error:", np.sum(np.not_equal(prediction,ytest))/len(ytest) )
print("Generalization score:", digits_nbc.score(Xtest,ytest))
print("Confusion matrix:")
print(confusion_matrix(ytest, prediction))

(1000, 64)
(1000,)
Generalization error: 0.18695106649937265
Generalization score: 0.8130489335006273
Confusion matrix:
[[83  0  0  0  0  1  0  0  0  0]
 [ 0 75  1  0  0  0  0  1  2  4]
 [ 0 10 57  0  1  0  1  0 14  0]
 [ 0  1  1 65  0  5  0  3  8  3]
 [ 1  3  0  0 55  0  3 18  0  0]
 [ 0  1  0  1  0 76  1  2  2  0]
 [ 0  2  0  0  0  0 67  0  0  0]
 [ 0  0  1  0  3  2  0 75  0  0]
 [ 0 17  1  1  0  2  0  6 46  0]
 [ 0  4  0  3  1  5  0  6  7 49]]


### Les colonnes montrent les vrais observations
### Les lignes montrent les observations prédits

In [31]:
# Compute cross-validation score
nb_trials = 20
score = []
for i in range(nb_trials):
    Xtrain, ytrain, Xtest, ytest = shuffle_and_split(X,y,1000)
    digits_nbc = GaussianNB()
    digits_nbc.fit(Xtrain,ytrain)
    score += [digits_nbc.score(Xtest,ytest)]
    print('*',end='')
print(" done!")
    
print("Average generalization score:", np.mean(score))
print("Standard deviation:", np.std(score))

******************** done!
Average generalization score: 0.8337515683814303
Standard deviation: 0.02045785098717688


Naive Bayes classifiers reach their limits on data with high correlations between features (like images).